# Reading and cleaning the data

In this notebook, we will read the Comma Separated Values (CSV) files. First, as nested matrices and secondly as the "inverted table" (columnar) database format.

In [3]:
]Import LINK ../link
LINK.Setup '#.EC' '../APLSource'
⎕CS EC

Imported: #.LINK ← C:\g\ECommerce\link

Linked: #.EC ←→ C:\g\ECommerce\APLSource

The Data Table object type is a namespace with members:
- `header` vector of column names
- `data` vector of column data
    - character data are matrices
    - numeric data are vectors
- `Column` function returns one or more columns of data by name

In [35]:
∇ ImportDataTable←{
  r←⎕NS ⍬   ⍝ Empty namespace
  _←r.⎕DF'[Data Table]'   ⍝ Data type label
  ⍺←1   ⍝ Read all data as characters by default
  r.(data header)←⎕CSV ⍵ ⍬ ⍺ 1 
  r.Column←{data[;header⍳⍥⎕C⊆⍵]}   ⍝ Look up column names case insensitively
  r
}
∇

In [4]:
]LINK.Add ImportDataTable

Added: #.EC.ImportDataTable

In [5]:
customers←ImportDataTable'../data/olist_customers_dataset.csv'

## ReadCSV
Some points of note:

- Because dfns finish execution on the first expression which is not an assignment (`name←expression`), we assign the result of `⎕DF` to underscore (`_`) as a "sink" or "throwaway" variable.
- Assignment to `⍺` within a dfn creates an ambivalent function. When `ReadCSV` is called monadically, the left argument `⍺` is `1`.
- The meaning of numeric column types specified by `⍺` is found in [the documentation for `⎕CSV`](http://help.dyalog.com/18.2/#Language/System%20Functions/csv.htm).

### Text as arguments
We obtain data by column name, rather than number, for better readability. To do so, we look up column names in the header vector:

In [10]:
customers.header⍳'customer_city'

6 6 6 6 6 6 6 6 6 6 6 6 6

The *index-of* primitive returns `1+≢⍺` where items in `⍵` are not found in `⍺`. In this case, it is looking for individual characters. We want to look up the entire string, but our lookup array `⍺` and search terms `⍵` have different structures.

`customers.header` is a nested array, as indicated by surrounding lines with `]Box on` or using `]Display`. More precisely, the depth `≡⍵` is greater than 1.

See `]display -?` and `]box -?` for more information about the boxed display of arrays.

In [18]:
]display customers.header
'Depth: ',≡customers.header

┌→───────────────────────────────────────────────────────────────────────────────────────────────┐
│ ┌→──────────┐ ┌→─────────────────┐ ┌→───────────────────────┐ ┌→────────────┐ ┌→─────────────┐ │
│ │customer_id│ │customer_unique_id│ │customer_zip_code_prefix│ │customer_city│ │customer_state│ │
│ └───────────┘ └──────────────────┘ └────────────────────────┘ └─────────────┘ └──────────────┘ │
└∊───────────────────────────────────────────────────────────────────────────────────────────────┘

Depth: 2

In contrast, `'customer_city'` is a **simple** (non-nested) character vector.

In [17]:
]display 'customer_city'
'Depth: ',≡'customer_city'

┌→────────────┐
│customer_city│
└─────────────┘

Depth: 1

## Data profile
Let's take a generic view which can be applied to any of our data sets.

In [32]:
∇ Report←{
⍝ ⍵: data table namespace
  parts←'Columns' 'Row count' 'Empty cell count' 'Row with empty cell count'
  parts,←'Duplicate row count' 'Columns with empty cells'
  parts,←'Total size in memory (bytes)' 'Average record size in memory (bytes)'
  r←⊂⍵.header ⋄  r,←≢⍵.data
  mt←(0∊⍴)¨⍵.data
  r,←+/,mt ⋄ r,←+/∨/mt ⋄ r,←+/~≠⍵.data ⋄ r,←⊂⍵.header⌿⍨∨⌿mt
  r,←⍬⍴⎕SIZE'⍵'
  r,←⊃(+⌿÷1⌈≢){row←⍵ ⋄ ⎕size'row'}¨↓⍵.data
  parts,⍪r
}
∇

In [33]:
]LINK.Add Report

Added: #.EC.Report

In [34]:
Report customers

┌─────────────────────────────────────┬──────────────────────────────────────────────────────────────────────────────────────┐
│Columns │┌───────────┬──────────────────┬────────────────────────┬─────────────┬──────────────┐│
│ ││customer_id│customer_unique_id│customer_zip_code_prefix│customer_city│customer_state││
│ │└───────────┴──────────────────┴────────────────────────┴─────────────┴──────────────┘│
├─────────────────────────────────────┼──────────────────────────────────────────────────────────────────────────────────────┤
│Row count │99441 │
├─────────────────────────────────────┼──────────────────────────────────────────────────────────────────────────────────────┤
│Empty cell count │0 │
├─────────────────────────────────────┼──────────────────────────────────────────────────────────────────────────────────────┤
│Row with empty cell count │0 │
├─────────────────────────────────────┼──────────────────────────────────────────────────────────────────────────────────────┤
│Duplicate row count │0 │
├─────────────────────────────────────┼──────────────────────────────────────────────────────────────────────────────────────┤
│Columns with empty cells │ │
│ │ │
│ │ │
├─────────────────────────────────────┼──────────────────────────────────────────────────────────────────────────────────────┤
│Total size in memory (bytes) │29221648 │
├─────────────────────────────────────┼──────────────────────────────────────────────────────────────────────────────────────┤
│Average record size in memory (bytes)│325.7601191 │
└─────────────────────────────────────┴──────────────────────────────────────────────────────────────────────────────────────┘

## Cleaning the data
Here we will inspect some data. We will remove rows containing empty values. 

We will look at the distributions of data

In [35]:
orders ← ImportDataTable'../data/olist_orders_dataset.csv'
Report orders

┌─────────────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│Columns │┌────────┬───────────┬────────────┬────────────────────────┬─────────────────┬────────────────────────────┬─────────────────────────────┬─────────────────────────────┐│
│ ││order_id│customer_id│order_status│order_purchase_timestamp│order_approved_at│order_delivered_carrier_date│order_delivered_customer_date│order_estimated_delivery_date││
│ │└────────┴───────────┴────────────┴────────────────────────┴─────────────────┴────────────────────────────┴─────────────────────────────┴─────────────────────────────┘│
├─────────────────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Row count │99441 │
├─────────────────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Empty cell count │4908 │
├─────────────────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Row with empty cell count │2980 │
├─────────────────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Duplicate row count │0 │
├─────────────────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Columns with empty cells │┌─────────────────┬────────────────────────────┬─────────────────────────────┐ │
│ ││order_approved_at│order_delivered_carrier_date│order_delivered_customer_date│ │
│ │└─────────────────┴────────────────────────────┴─────────────────────────────┘ │
├─────────────────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Total size in memory (bytes) │51585160 │
├─────────────────────────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│Average record size in memory (bytes)│550.6502951 │
└─────────────────────────────────────┴───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

In [17]:
'rows, columns:',⍴orders.data
mt←(0∊⍴)¨orders.data   ⍝ Find empty cells
'Count of empty cells:',+/,mt
'Count of rows with empty cells:',+/∨/mt
'Columns with empty cells:'
orders.header⌿⍨∨⌿mt    

rows, columns: 99441 8

Count of empty cells: 4908

Count of rows with empty cells: 2980

Columns with empty cells:

┌─────────────────┬────────────────────────────┬─────────────────────────────┐
│order_approved_at│order_delivered_carrier_date│order_delivered_customer_date│
└─────────────────┴────────────────────────────┴─────────────────────────────┘

99441 5